# ChatGLM3 Lora 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/home/intel/miniforge3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/home/intel/.cache/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/91a0561caa089280e94bf26a9fc3530482f0fe60", trust_remote_code=True)
tokenizer

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


ChatGLMTokenizer(name_or_path='/home/intel/.cache/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/91a0561caa089280e94bf26a9fc3530482f0fe60', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	64790: AddedToken("[gMASK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	64792: AddedToken("sop", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	64795: AddedToken("<|user|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	64796: AddedToken("<|assistant|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [5]:
tokenizer(tokenizer.eos_token), tokenizer.eos_token_id

({'input_ids': [64790, 64792, 2893, 30917, 30994], 'attention_mask': [1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4]},
 2)

In [6]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = "\n".join([example["instruction"], example["input"]]).strip()     # query
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n query<|assistant|>
    response = tokenizer("\n" + example["output"], add_special_tokens=False)        # \n response, 缺少eos token
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK] sop <|user|> \n 解释为什么以下分数等同于1/4\n输入：4/16 <|assistant|> \n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'\n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

## Step4 创建模型

In [10]:
import torch
# 多卡情况，可以去掉device_map="auto"，否则会将模型拆开，导致训练出问题
model = AutoModelForCausalLM.from_pretrained("/home/intel/.cache/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/91a0561caa089280e94bf26a9fc3530482f0fe60", trust_remote_code=True, low_cpu_mem_usage=True, 
                                             torch_dtype=torch.bfloat16, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16,
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.33it/s]


In [11]:
for name, param in model.named_parameters():
    print(name, param.dtype)

transformer.embedding.word_embeddings.weight torch.bfloat16
transformer.encoder.layers.0.input_layernorm.weight torch.bfloat16
transformer.encoder.layers.0.self_attention.query_key_value.weight torch.uint8
transformer.encoder.layers.0.self_attention.query_key_value.bias torch.bfloat16
transformer.encoder.layers.0.self_attention.dense.weight torch.uint8
transformer.encoder.layers.0.post_attention_layernorm.weight torch.bfloat16
transformer.encoder.layers.0.mlp.dense_h_to_4h.weight torch.uint8
transformer.encoder.layers.0.mlp.dense_4h_to_h.weight torch.uint8
transformer.encoder.layers.1.input_layernorm.weight torch.bfloat16
transformer.encoder.layers.1.self_attention.query_key_value.weight torch.uint8
transformer.encoder.layers.1.self_attention.query_key_value.bias torch.bfloat16
transformer.encoder.layers.1.self_attention.dense.weight torch.uint8
transformer.encoder.layers.1.post_attention_layernorm.weight torch.bfloat16
transformer.encoder.layers.1.mlp.dense_h_to_4h.weight torch.uint8


## Lora

### PEFT Step1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(target_modules=["query_key_value"])
config

### PEFT Step2 创建模型

In [14]:
model = get_peft_model(model, config)

In [ ]:
config

In [16]:
model.enable_input_require_grads()

In [ ]:
model.print_trainable_parameters()

## Step5 配置训练参数

In [18]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"
)

## Step6 创建训练器

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [ ]:
trainer.train()

In [12]:
#trainer.save_model()
model.save_pretrained("./chatbot")
tokenizer.save_pretrained("./chatbot")

('./chatbot/tokenizer_config.json',
 './chatbot/special_tokens_map.json',
 './chatbot/tokenizer.model',
 './chatbot/added_tokens.json')

## Step8 模型推理

In [13]:
model.eval()
print(model.chat(tokenizer, "数学考试怎么考高分？", history=[])[0])

想要在数学考试中获得高分，需要掌握以下几点：

1. 扎实的基本功：数学考试要求熟练掌握基础知识，如加减乘除、分数、百分数、三角函数、代数等。基础扎实才能在考试中迅速解题。

2. 解题技巧：掌握解题技巧和方法，如先易后难、逻辑推理、归纳法、记忆方法等。通过多练习，熟练运用这些技巧，提高解题速度和正确率。

3. 题目分析：在考试中，题目分析很重要。仔细阅读题目，理解题目要求，找出解题的关键点，制定解题策略。

4. 时间分配：合理分配考试时间，先做会做的题目，不会做的题目暂且放慢，合理分配时间，避免因时间紧张导致失分。

5. 复习和练习：定期进行复习和练习，查漏补缺。多做习题，总结经验，巩固基础知识。

6. 考试心态：保持积极的心态，考试过程中保持冷静，遇到难题不慌张。相信自己，尽力而为。

7. 及时总结：每次考试后，总结自己的错误和不足，找出原因，针对性地进行改进。

8. 合理安排学习时间：数学需要较强的逻辑性，需要花费一定的时间来理解和掌握。要合理安排学习时间，保证数学学习的时间和质量。

只要努力学习，掌握方法，相信你会在数学考试中取得好成绩。


In [2]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/home/intel/project/quantize/transformer/chatbot"


# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
print("Tokenizer loaded successfully.")

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
print("Model loaded successfully.")

/home/intel/miniforge3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Tokenizer loaded successfully.
Model loaded successfully.


In [4]:
model.eval()
print(model.chat(tokenizer, "介绍北京？", history=[])[0])

北京，简称“京”，是中华人民共和国的首都、直辖市、国家中心城市、超大城市，国务院首批沿海开放城市。位于华北地区，背靠燕山，毗邻天津市和河北省。全市总面积1.6万平方公里，常住人口约为2170万（截至2020年末）。

北京是我国的政治、文化、科技创新、国际交往和综合交通枢纽中心，拥有世界著名的文化古迹和自然景观。自古以来就是著名的历史文化名城，有着3000多年的历史，是元、明、清三个朝代的都城。北京有着丰富的旅游资源，包括故宫、长城、颐和园、天坛、北海公园等世界文化遗产，以及天安门、颐和园、圆明园等著名景点。

北京也是我国的教育和科研重镇，拥有清华大学、北京大学等著名高校，以及中国科学院、中国科学技术院等国家级科研机构。此外，北京还是2022年冬奥会的举办地，届时将成为全球关注的焦点。

作为中国的一个重要城市，北京经济实力雄厚，是我国GDP排名前列的城市之一。近年来，北京致力于发展现代服务业、高科技产业和创新产业，打造国际化、法治化、现代化的都市。同时，北京也在努力改善生态环境，推动城市可持续发展。


In [8]:
!pip list|grep transformers
!pip list|grep tokenizer
!pip list

transformers              4.37.2
tokenizers                0.15.2
Package                   Version
------------------------- ------------
accelerate                0.26.1
aiofiles                  23.2.1
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
altair                    5.4.1
annotated-types           0.7.0
anyio                     4.6.2.post1
asttokens                 2.4.1
async-timeout             4.0.3
attrs                     24.2.0
audioread                 3.0.1
auto-gptq                 0.6.0
autoawq                   0.2.2
autoawq_kernels           0.0.7
bitsandbytes              0.41.3.post2
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
click                     8.1.7
coloredlogs               15.0.1
comm                      0.2.2
contourpy                 1.3.0
cycler                    0.12.1
dataclasses-json          0.6.7
datasets                  2.16.1
deb